# Homework 1: Regression

Data is air quality, needs to predict the PM2.5 in other times/ other situation.
raw data:
columns means each hour of the day

following steps:
1. load data, and view the features
2. reorgnize data, and clean data, divide into testing and validation set
3. build a model
4. test the loss function
5. use the best model to train through the whole data.

### load data, and see

In [1]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('./Documents/LeeML-datasets/hw1/train.csv')

In [2]:
# data.head(18)
# notice that RAINFALL needs to be trans into numbers 

In [3]:
data = data.iloc[:, 3:] # make datas begin with column 3
data[data == 'NR'] = 0
raw_data = data.to_numpy()
#data.head(18)

From containt of slides, every day's data will also be treat as a sample, so there will be 18(features) * 480(hours) per month.
using month_data to store 18 * 480 hours data

In [4]:
month_data = {}
for month in range(12):
    sample = np.empty([18,480])#create an empty matrics
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample
# a;b is slicing the data from 24 to 48(day = 1)
# ; alone means all the data
month_data[0]

array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]])

#### hard to comprehend:
using 10 hours as a group, using former 9 hours as features to predict the 10th hours PM2.5.
every month has 480 hours, each 9 hours form a new data, every month has 471 datas.
So each set of data has 9 * 18 features.
And the matching target has 471(10th hour data) * 12(month). 

In [5]:
x = np.empty([12*471, 18*9], dtype = float)
y = np.empty([12*471,1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:, day * 24 + hour : day* 24 + hour + 9].reshape(1,-1)
            '''reshape function transforms the matrics into vectors'''
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day* 24 + hour + 9]#pick up pm2.5 data
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


#### Normalize:
make all the data, in matrics, k become (k - mean_k)/ standard_deviation_k

In [6]:
mean_x = np.mean(x, axis = 0)# through 18 * 9, axis == 0 means based on rows 
std_x = np.std(x, axis =0)# through 18 * 9, has only one column, 471 elements
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0: # if not all the same
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(x)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


### Divide into train_set and validation_set
first using the easy way(20210425)
second using the N-fold cross validation

In [7]:
import math
# divide into 8:2
# floor means max integer that is no bigger than x. floor(3.5)=3
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8) :, :]
y_validation = y[math.floor(len(y) * 0.8) :, :]
print(len(x_train_set))
print(x_train_set)
print(len(x_validation))
print(x_validation)

4521
[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
1131
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]


### Training
model function: ty = bias + weight_vec * x_vec 

because I normalized the data before, so bias will be 0, and can be ignored

loss function: loss = sum of (y - ty)^2

(concatenate: connect)

In [8]:
# numpy.dot(a,b),product of two array
# numpy.concatenate, join a sequence of arrays along an existing axis,
# axis = 0, add extra rows, axis = 1, add some columns behind
'''
pseudocode:
w = np.zeros([len(x[0]),1])
ty[i] = np.dot(w, x[i])
loss = np.sum(np.power(ty - y))
'''


'\npseudocode:\nw = np.zeros([len(x[0]),1])\nty[i] = np.dot(w, x[i])\nloss = np.sum(np.power(ty - y))\n'

some tips from answers:
1. bias is putted into weight array with x^0
2. eps is for avoid if adagrad is 0
every dimension has its own gradient, weight, using iteration to learn and get close to the good model.

### preserve for analyze (code)
dim = len(x[0]) + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([len(x),1]), x), axis = 1).astype(float) 
#add extra column for bias to calculate

learning_rate = 100
#10 : 5.74 20 : 5.75 50 : 5.81 100 : 6.07
iter_time = 2000 
#1000: loss 7.336, 2000: loss 6.07
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)
    #root mean square error
    #the_lambda = 10 # using this to make function smooth
    #loss = np.sum(np.power(np.dot(x, w) - y), 2) + np.sum(w)*the_lambda
    #square error
    if(t%100 == 0):
        print(str(t) + " : " + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('easy_weight.npy', w)
#print(w)

In [9]:
dim = len(x_train_set[0]) + 1
w1 = np.zeros([dim, 1])
x_train_set = np.concatenate((np.ones([len(x_train_set),1]), x_train_set), axis = 1).astype(float) 
# add extra column for bias to calculate
# this kind of changing should run just one tims....

learning_rate = 100
iter_time = 1000 

adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w1) - y_train_set, 2))/471/12)
    # root mean square error
    # the_lambda = 10 # using this to make function smooth
    # loss = np.sum(np.power(np.dot(x, w) - y), 2) + np.sum(w)*the_lambda
    # square error
    if(t%100 == 0 or t==999):
        print(str(t) + " : " + str(loss))
    gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w1) - y_train_set)
    adagrad += gradient ** 2
    w1 = w1 - learning_rate * gradient / np.sqrt(adagrad + eps)
print(len(x_train_set[0]))
#print(w1)

0 : 24.36221492236629
100 : 20.323593225645215
200 : 9.326264559340517
300 : 6.829236149377322
400 : 6.1263112412977305
500 : 5.829997137753871
600 : 5.657451772718693
700 : 5.54002913452329
800 : 5.454822077702424
900 : 5.391104701661769
999 : 5.34297576023664
163


### using validation data to analyze
originally, using N times validation test to find the best model.
in this section, I will just use one to test the difference of loss
between train set and validation set

In [10]:
x_validation = np.concatenate((np.ones([len(x_validation),1]), x_validation), axis = 1).astype(float) 
predict_y = np.dot(x_validation, w1)
loss = np.sqrt(np.sum(np.power(predict_y - y_validation, 2))/471/0.8/12)
print(loss)
# OMG,this model is better in validation test set...

2.9568871573930027


### retrain my model with this weight?
how to do that !!!!?????

##### my idea was that using this model and change the learing rate to a very small step, then change it,and get closer to the optimal point.

In [11]:
# messed up down below
dim = len(x[0]) + 1
w = w1
x = np.concatenate((np.ones([len(x),1]), x), axis = 1).astype(float) 

learning_rate = 2
iter_time = 1000 

adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)
    # root mean square error
    if(t%100 == 0 or t==999):
        print(str(t) + " : " + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)


0 : 5.961324435412296
100 : 6.051726096749511
200 : 5.872714066229467
300 : 5.816683595388648
400 : 5.78633117026658
500 : 5.766227421465352
600 : 5.7516478392706665
700 : 5.740528396384372
800 : 5.731768978091741
900 : 5.724711033823115
999 : 5.718981858877895


### Testing the test data
use the former approaches to get the test data, and get the result store it into a csv file.

In [12]:
np.save('./Documents/LeeML-datasets/hw1/my_weight.npy', w)

In [13]:
testdata = pd.read_csv('./Documents/LeeML-datasets/hw1/test.csv', header = None, encoding = 'big5')
# because this data has no header!!! which made me confused a lot
#testdata.head(18)

In [14]:
testdata = testdata.iloc[:, 2:]
testdata[testdata == 'NR'] = 0
test_data = testdata.to_numpy()
testdata.head()

,2,3,4,5,6,7,8,9,10
0,21,21,20,20,19,19,19,18,17
1,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [15]:
test_x = np.empty([240, 18*9], dtype = float)
# change the shape
# print(test_data[18 : 36, :])
for i in range(240):
    test_x[i, :] = test_data[18 * i : 18 * (i + 1), :].reshape(1,-1)
#print(test_x[1,:])

# normalize the data
mean_x = np.mean(test_x, axis = 0)
std_x = np.std(test_x, axis =0)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if (std_x[j] != 0):
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]

test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)

### Prediction

In [16]:
w = np.load('./Documents/LeeML-datasets/hw1/my_weight.npy')
predict_y = np.dot(test_x, w)
predict_y

array([[ 4.5431476 ],
       [14.02117361],
       [18.09343536],
       [ 5.25913983],
       [20.74423887],
       [16.1286735 ],
       [17.8913089 ],
       [23.58778635],
       [13.27326597],
       [45.37830155],
       [ 9.33061147],
       [ 7.25056658],
       [48.96288145],
       [39.90567047],
       [15.77336505],
       [ 8.66010711],
       [25.32467419],
       [52.65533054],
       [-2.67354193],
       [12.40161989],
       [33.07237337],
       [56.70869938],
       [ 7.2426794 ],
       [14.51902912],
       [10.87441237],
       [28.84708081],
       [11.00496847],
       [51.45482206],
       [ 5.63783555],
       [42.52577613],
       [18.08610727],
       [ 5.34151965],
       [ 2.54001343],
       [14.41149124],
       [21.60734647],
       [28.95935514],
       [33.68054406],
       [22.57225256],
       [34.19633099],
       [26.97107102],
       [ 4.79789269],
       [32.30756041],
       [24.21979122],
       [39.22780501],
       [13.34211133],
       [26

### Save into CSV file

##### I just copied all the code,,,below. skip this step for a little break...

In [17]:
import csv
with open('./Documents/LeeML-datasets/hw1/my_submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), predict_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 4.543147595856026]
['id_1', 14.021173610470026]
['id_2', 18.093435360716352]
['id_3', 5.259139825510993]
['id_4', 20.744238868464116]
['id_5', 16.128673504243633]
['id_6', 17.891308896226676]
['id_7', 23.587786345507265]
['id_8', 13.27326597222266]
['id_9', 45.37830155351644]
['id_10', 9.330611467980622]
['id_11', 7.250566579908986]
['id_12', 48.96288145311485]
['id_13', 39.90567047007214]
['id_14', 15.7733650540283]
['id_15', 8.660107105271452]
['id_16', 25.324674194682135]
['id_17', 52.65533053647614]
['id_18', -2.673541932344774]
['id_19', 12.40161989186005]
['id_20', 33.072373373405306]
['id_21', 56.70869938164353]
['id_22', 7.242679395478758]
['id_23', 14.5190291242052]
['id_24', 10.874412366860275]
['id_25', 28.84708081158503]
['id_26', 11.004968465040438]
['id_27', 51.4548220609149]
['id_28', 5.6378355473803285]
['id_29', 42.525776125036685]
['id_30', 18.08610726989456]
['id_31', 5.341519650252842]
['id_32', 2.540013428245686]
['id_33', 14.41149124467803